In [ ]:
import pandas as pd, itertools

def process_data(data):

    df_year_and_quarter = pd.read_csv(data)
    
    #Make a list of item column names
    columns_as_string = "Item 1	Item 2	Item 3	Item 4	Item 5	Item 6	Item 7	Item 8	Item 9	Item 10	Item 11	Item 12	Item 13	Item 14	Item 15	Item 16	Item 17	Item 18	Item 19	Item 20	Item 21	Item 22	Item 23	Item 24	Item 25	Item 26	Item 27	Item 28	Item 29	Item 30	Item 31	Item 32"
    columns_as_list = columns_as_string.split('	')

    #Only include rows where item count is two or more
    processed_df= df_year_and_quarter.loc [ df_year_and_quarter ["Items(s)"] > 2]

    #Make a list of combinations of each column
    item_column_combinations = itertools.combinations(columns_as_list, 2)
    
    #Make new columns for each possible column combination
    for column_a, column_b in item_column_combinations:
        processed_df [f'{column_a}_and_{column_b}_pair'] = processed_df [f'{column_a}'].astype(str) + '-' + processed_df [f'{column_b}'].astype(str)

    #for i in range(len(itertools.combinations(processed_df.columns, 2))):
        #for j in range(len(processed_df.columns)):
            #for a, b in itertools.combinations(processed_df.columns, 2)
                #processed_df [f'item_{a}_and_{b}_pair'] = processed_df [f'{a}'].astype(str) + '-' + processed_df [f'{b}'].astype(str)
    
    #Drop individual item columns, leaving only our item combo columns
    processed_df.drop(columns = columns_as_list, inplace=True)
    
    #Make a list of the actual unique items in existing columns
    unique_items = []
    for i in columns_as_list:
        unique_items.append(processed_df[f'{i}'].unique())
    
    return processed_df, unique_items

    

def check (processed_df, column_name, item_a, item_b):
    return len (processed_df [ processed_df [ column_name ] == str(item_a)+ '-' + str(item_b)]) == 0 and len(processed_df [ processed_df [ column_name  ] == str(item_b) + '-' + str(item_a)]) == 0:
    


data = "C:\Users\isaac\Downloads\groceries - groceries.csv"
process_data(data)

#After running process data function, weshould have unique item list and a processed df.
#Now, for each possible combination of unique items, we will check each column

for item_a, item_b in itertools.combinations(unique_items, 2):
    for column_name in process_data.columns:
        # Flag to track if (a, b) is found in all sources
        all_not_found = True
        if check(processed_df, column_name, item_a, item_b):
            # Check if (a, b) exists in the current dataset

            # If check returns True, continue checking other sources
            continue
        else:
            # If check returns False, set flag to False and break
            all_not_found = False
            break

    # If the pair (a, b) is not found in any dataset, print the message
    if all_not_found:
        print(f"{item_a}, {item_b} not found in data")
        
    
